In [40]:
from pathlib import Path
import pandas as pd

from rekall.predicates import _iou, overlaps
from virat_helper import load_events, LIST_CLIP_ID, EVENTTYPE

In [2]:
print(LIST_CLIP_ID)

['VIRAT_S_010000_08_000893_001024', 'VIRAT_S_010000_03_000442_000528', 'VIRAT_S_010201_02_000167_000197', 'VIRAT_S_010003_01_000111_000137', 'VIRAT_S_050000_11_001530_001576', 'VIRAT_S_010202_06_000784_000873', 'VIRAT_S_000201_01_000384_000589', 'VIRAT_S_010113_07_000965_001013', 'VIRAT_S_050000_06_000908_000970', 'VIRAT_S_000201_06_001354_001397', 'VIRAT_S_010203_02_000347_000397', 'VIRAT_S_010201_08_000705_000739', 'VIRAT_S_010205_02_000301_000345', 'VIRAT_S_010203_00_000047_000139', 'VIRAT_S_010109_04_000487_000538', 'VIRAT_S_010107_02_000282_000312', 'VIRAT_S_040001_02_001102_001530', 'VIRAT_S_050203_06_001202_001264', 'VIRAT_S_040103_07_001011_001093', 'VIRAT_S_010003_09_000779_000861', 'VIRAT_S_010001_04_000583_000646', 'VIRAT_S_050000_08_001235_001295', 'VIRAT_S_040003_01_000083_000193', 'VIRAT_S_010111_09_000981_001014', 'VIRAT_S_010113_06_000831_000896', 'VIRAT_S_050300_08_001705_001771', 'VIRAT_S_050300_05_001272_001330', 'VIRAT_S_010200_06_000702_000744', 'VIRAT_S_040000_05_

In [42]:
def parse_event_list(clip_id, event_type) -> pd.DataFrame:
    try:
        df = load_events(clip_id)
    except FileNotFoundError:
        # some clips have no events and no event file
        return pd.DataFrame()
    
    df = df[df['event_type']==event_type]
    df['x2'] = df['x1'] + df['w']
    df['y2'] = df['y1'] + df['h']
    df['t1'], df['t2'] = df['start_frame'], df['end_frame']
    
    rv = []
    for event_id in df['event_id'].unique():
        edf = df[df['event_id']==event_id]
        rv.append({
            'event_id': event_id,
            'x1': edf['x1'].min(),
            'x2': edf['x2'].max(),
            'y1': edf['y1'].min(),
            'y2': edf['y2'].max(),
            't1': edf['t1'].min(),
            't2': edf['t2'].max(),
        })
        
    return  pd.DataFrame(rv)

def parse_all_event_list(event_type) -> pd.DataFrame:
    all_df = []
    for clip_id in LIST_CLIP_ID:
        df = parse_event_list(clip_id, event_type)
        if not df.empty:
            df['clip_id'] = clip_id
            all_df.append(df)
        
    return pd.concat(all_df, ignore_index=True)
    

# gt = parse_event_list('VIRAT_S_000202_00_000000_000977', EVENTTYPE.GET_OUT_VEHICLE)
GT = parse_all_event_list(EVENTTYPE.GET_OUT_VEHICLE)
print(GT)

    event_id   x1    x2   y1   y2    t1    t2                          clip_id
0         10  383   546  146  331  1135  1271  VIRAT_S_050000_11_001530_001576
1          0  695   767  362  420   631  1022  VIRAT_S_000201_01_000384_000589
2          0  132   213  352  435   449   701  VIRAT_S_000201_06_001354_001397
3          0  686   786  353  434   572   758  VIRAT_S_000201_03_000640_000672
4          2  423   609  509  690  3340  3517                   VIRAT_S_000006
..       ...  ...   ...  ...  ...   ...   ...                              ...
92         0  786  1034  501  711  2486  2619                   VIRAT_S_000002
93         0  529   643   58  117   656   745  VIRAT_S_050203_01_000147_000223
94         1  454   583   84  154  1067  1125  VIRAT_S_050203_01_000147_000223
95         0  660   771  475  515   637   832  VIRAT_S_000200_00_000100_000171
96         0  685   767  336  379  1180  1296  VIRAT_S_000203_07_001341_001458

[97 rows x 8 columns]


In [43]:
def parse_result(result_file="getoutcar.csv"):
    df = pd.read_csv(result_file, index_col=0)
    # convert relative coord to pixel coord
    df['x1'] = df['x1'] * df['width']
    df['x2'] = df['x2'] * df['width']
    df['y1'] = df['y1'] * df['height']
    df['y2'] = df['y2'] * df['height']
    return df

prediction = parse_result()
print(prediction)

                             clip_id    t1    t2           x1           x2  \
0    VIRAT_S_000201_01_000384_000589   810  1140   707.153931   806.214981   
1    VIRAT_S_000203_04_000903_001086  3930  4260   861.986084   933.536606   
2    VIRAT_S_000203_04_000903_001086  4620  4950   494.952393   632.016563   
3                     VIRAT_S_000004  7740  8070   755.062695  1037.355080   
4                     VIRAT_S_000006  3600  3930   398.942928   748.659725   
..                               ...   ...   ...          ...          ...   
346  VIRAT_S_050300_04_001057_001122   900  1230   733.956242   865.514774   
347  VIRAT_S_050301_01_000083_000320  1350  1680   699.776688   846.244640   
348  VIRAT_S_050301_01_000083_000320  2640  2970   693.724880   862.611580   
349  VIRAT_S_050301_01_000083_000320  5940  6270  1409.059525  1506.214234   
350  VIRAT_S_050301_01_000083_000320  6900  7090  1412.147255  1499.964981   

             y1          y2  frame_count  fps  width  height  


In [60]:
def is_hit(i1, i2):
    return _iou(i1, i2) > 0.01 and overlaps()(i1, i2)

In [61]:
# Evaluate precision and recall
GT['hit'] = False
prediction['hit'] = False

for i, pred in prediction.iterrows():
    for j, gt in GT.iterrows():
        if pred['clip_id'] == gt['clip_id'] and is_hit(pred, gt):
            prediction.loc[i, ['hit']] = True
            GT.loc[j, ['hit']] = True
            
print(GT)


    event_id   x1    x2   y1   y2    t1    t2  \
0         10  383   546  146  331  1135  1271   
1          0  695   767  362  420   631  1022   
2          0  132   213  352  435   449   701   
3          0  686   786  353  434   572   758   
4          2  423   609  509  690  3340  3517   
..       ...  ...   ...  ...  ...   ...   ...   
92         0  786  1034  501  711  2486  2619   
93         0  529   643   58  117   656   745   
94         1  454   583   84  154  1067  1125   
95         0  660   771  475  515   637   832   
96         0  685   767  336  379  1180  1296   

                            clip_id    hit  
0   VIRAT_S_050000_11_001530_001576  False  
1   VIRAT_S_000201_01_000384_000589   True  
2   VIRAT_S_000201_06_001354_001397  False  
3   VIRAT_S_000201_03_000640_000672  False  
4                    VIRAT_S_000006  False  
..                              ...    ...  
92                   VIRAT_S_000002  False  
93  VIRAT_S_050203_01_000147_000223  False  
94  VI

In [62]:
GT_hit = len(GT[GT['hit']==True])
prediction_hit = len(prediction[prediction['hit']==True])
print(f"Precision={prediction_hit}/{len(prediction)}={prediction_hit/len(prediction):.2f}")
print(f"Rekall={GT_hit}/{len(GT)}={GT_hit/len(GT):.2f}")

Precision=12/351=0.03
Rekall=11/97=0.11


In [63]:
print(prediction[prediction['hit']==True])

                             clip_id     t1     t2           x1           x2  \
0    VIRAT_S_000201_01_000384_000589    810   1140   707.153931   806.214981   
10   VIRAT_S_000207_01_000094_000156    690   1020   534.324799   630.192337   
11   VIRAT_S_000207_01_000094_000156    780   1110   534.324799   630.192337   
78   VIRAT_S_000202_00_000000_000977  25410  25740   613.396683   699.948120   
183  VIRAT_S_040000_09_001194_001574   1230   1560   856.638851  1008.090706   
191  VIRAT_S_040103_03_000284_000425    720   1050   471.185017   681.561699   
218  VIRAT_S_040104_02_000459_000721   1020   1350  1047.671700  1276.631012   
219  VIRAT_S_040104_02_000459_000721   1110   1440  1047.671700  1276.631012   
220  VIRAT_S_040104_02_000459_000721   2100   2430   451.911160   768.133965   
221  VIRAT_S_040104_02_000459_000721   2100   2430   459.570894   768.133965   
256  VIRAT_S_050203_09_001960_002083   1110   1440   556.675930   842.501678   
257  VIRAT_S_050203_09_001960_002083   1